In [1]:
# Image Denoising: Linear (Mean) vs Nonlinear (Median) Filter

import cv2
import numpy as np
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from matplotlib import pyplot as plt

# Create output directories
os.makedirs('output/gaussian', exist_ok=True)
os.makedirs('output/salt_pepper', exist_ok=True)
os.makedirs('output/speckle', exist_ok=True)

# Load image
def load_image(path, gray=True):
    img = cv2.imread(path)
    if gray:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

# Add noise
def add_noise(img, noise_type):
    if noise_type == 'gaussian':
        mean = 0
        var = 20
        sigma = var ** 0.5
        gauss = np.random.normal(mean, sigma, img.shape).reshape(img.shape)
        noisy = img + gauss
        return np.clip(noisy, 0, 255).astype(np.uint8)

    elif noise_type == 'salt_pepper':
        prob = 0.05
        noisy = np.copy(img)
        rnd = np.random.rand(*img.shape)
        noisy[rnd < prob] = 0
        noisy[rnd > 1 - prob] = 255
        return noisy

    elif noise_type == 'speckle':
        noise = np.random.randn(*img.shape)
        noisy = img + img * noise * 0.2
        return np.clip(noisy, 0, 255).astype(np.uint8)

# Apply filters
def mean_filter(img):
    return cv2.blur(img, (3, 3))

def median_filter(img):
    return cv2.medianBlur(img, 3)

# Evaluate performance
def evaluate(img_original, img_denoised):
    mse_val = np.mean((img_original - img_denoised) ** 2)
    psnr_val = psnr(img_original, img_denoised)
    ssim_val = ssim(img_original, img_denoised)
    return mse_val, psnr_val, ssim_val

# Main experiment
def run_experiment(image_path):
    original = load_image(image_path)
    noise_types = ['gaussian', 'salt_pepper', 'speckle']
    results = []

    for noise in noise_types:
        noisy = add_noise(original, noise)
        mean_denoised = mean_filter(noisy)
        median_denoised = median_filter(noisy)

        mse_mean, psnr_mean, ssim_mean = evaluate(original, mean_denoised)
        mse_median, psnr_median, ssim_median = evaluate(original, median_denoised)

        results.append({
            'noise': noise,
            'mean': {'MSE': mse_mean, 'PSNR': psnr_mean, 'SSIM': ssim_mean},
            'median': {'MSE': mse_median, 'PSNR': psnr_median, 'SSIM': ssim_median}
        })

        # Save results
        cv2.imwrite(f'output/{noise}/original.png', original)
        cv2.imwrite(f'output/{noise}/noisy.png', noisy)
        cv2.imwrite(f'output/{noise}/mean.png', mean_denoised)
        cv2.imwrite(f'output/{noise}/median.png', median_denoised)

    return results

# Run
if __name__ == '__main__':
    image_path = 'image.jpg'  # Replace with your own image path
    results = run_experiment(image_path)

    for res in results:
        print(f"Noise Type: {res['noise']}")
        print(f"  Mean Filter  -> MSE: {res['mean']['MSE']:.2f}, PSNR: {res['mean']['PSNR']:.2f}, SSIM: {res['mean']['SSIM']:.3f}")
        print(f"  Median Filter-> MSE: {res['median']['MSE']:.2f}, PSNR: {res['median']['PSNR']:.2f}, SSIM: {res['median']['SSIM']:.3f}\n")


C:\ProgramData\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


Noise Type: gaussian
  Mean Filter  -> MSE: 12.93, PSNR: 29.50, SSIM: 0.948
  Median Filter-> MSE: 8.04, PSNR: 37.69, SSIM: 0.957

Noise Type: salt_pepper
  Mean Filter  -> MSE: 54.86, PSNR: 22.05, SSIM: 0.381
  Median Filter-> MSE: 5.28, PSNR: 32.92, SSIM: 0.986

Noise Type: speckle
  Mean Filter  -> MSE: 45.83, PSNR: 25.92, SSIM: 0.719
  Median Filter-> MSE: 47.28, PSNR: 26.94, SSIM: 0.663

